In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.PowerShell,*-*"
#r "nuget:Microsoft.DotNet.Interactive.Http,*-*"
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive, *-*"
using Plotly.NET;

Installed Packages Microsoft.DotNet.Interactive.Http, 1.0.0-beta.22175.2 Microsoft.DotNet.Interactive.PowerShell, 1.0.0-beta.22175.2 Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.22175.2 Plotly.NET, 2.0.0-preview.18 Plotly.NET.Interactive, 2.0.0-preview.18

Loading extensions from `Plotly.NET.Interactive.dll`

Loading extensions from `Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [ ]:
#!connect mssql --kernel-name splSnapshots "Server=localhost;Database=SeattlePublicLibraryOpenData;Trusted_Connection=True;TrustServerCertificate=true;"

Error: System.ArgumentException: Alias '#!sql-splSnapshots' is already in use.
   at Microsoft.DotNet.Interactive.CompositeKernel.Add(Kernel kernel, IReadOnlyCollection`1 aliases) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\CompositeKernel.cs:line 91
   at Microsoft.DotNet.Interactive.CompositeKernel.<>c__DisplayClass29_0`1.<<AddKernelConnector>b__0>d.MoveNext() in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\CompositeKernel.cs:line 351
--- End of stack trace from previous location ---
   at System.CommandLine.NamingConventionBinder.CommandHandler.GetExitCodeAsync(Object returnValue, InvocationContext context)
   at System.CommandLine.NamingConventionBinder.ModelBindingCommandHandler.InvokeAsync(InvocationContext context)
   at System.CommandLine.Invocation.InvocationPipeline.<>c__DisplayClass4_0.<<BuildInvocationChain>b__0>d.MoveNext()
--- End of stack trace from previous location ---
   at System.CommandLine.Builder.CommandLineBuilderExtensions.<>c__DisplayClass17_0

In [ ]:

#!sql-splSnapshots --name locationDestributionOverTime

;WITH condensedbranchcode_cte
     AS (SELECT CASE code
                  WHEN 'lock1' THEN 'hip'
                  WHEN 'lock2' THEN 'rbe'
                  WHEN 'tcs' THEN 'cen'
                  ELSE code
                END                AS branchCode,
                Min([description]) AS [Description]
         FROM   itemlocation
         GROUP  BY CASE code
                     WHEN 'lock1' THEN 'hip'
                     WHEN 'lock2' THEN 'rbe'
                     WHEN 'tcs' THEN 'cen'
                     ELSE code
                   END),
     allreportdates_cte
     AS (SELECT DISTINCT reportdate,
                         branchcode,
                         [description]
         FROM   elementdetail
                CROSS JOIN condensedbranchcode_cte),
     condensedbranchinventory_cte
     AS (SELECT reportdate,
                branchcode,
                Sum(ei.itemcount) AS BranchItemCount
         FROM   elementinventory ei
                INNER JOIN condensedbranchcode_cte cbc
                        ON cbc.branchcode = ei.itemlocationcode
                INNER JOIN itemlocation il
                        ON il.code = branchcode
         GROUP  BY reportdate,
                   branchcode)
SELECT cbc.reportdate,
       cbc.branchcode,
       cbc.description,
       Isnull(cbi.branchitemcount, 0) AS BranchItemCount
FROM   allreportdates_cte cbc
       LEFT JOIN condensedbranchinventory_cte cbi
              ON cbi.branchcode = cbc.branchcode
                 AND cbc.reportdate = cbi.reportdate
ORDER  BY reportdate ASC,
          branchcode; 


(2176 rows affected)

reportdate,branchcode,description,BranchItemCount
2017-01-01 00:00:00Z,bal,Ballard Branch,59666
2017-01-01 00:00:00Z,bea,Beacon Hill Branch,38882
2017-01-01 00:00:00Z,bro,Broadview Branch,47568
2017-01-01 00:00:00Z,cap,Capitol Hill Branch,40565
2017-01-01 00:00:00Z,cen,Central Library,1028187
2017-01-01 00:00:00Z,col,Columbia Branch,34599
2017-01-01 00:00:00Z,dlr,Delridge Branch,18077
2017-01-01 00:00:00Z,drp1,*** Temporary ***,273
2017-01-01 00:00:00Z,dth,Douglass-Truth Branch,49929
2017-01-01 00:00:00Z,fre,Fremont Branch,18899


In [ ]:
#!share --from sql-splSnapshots locationDestributionOverTime
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

IList values = new List<object>();
IList keys = new List<object>();
List<GenericChart.GenericChart> charts = new();
//List<Bar> bars = new List<Bar>();
//var colors = new string[]{"#324489", "#3b3e81", "#44387a", "#4d3173", "#562b6c", "#5f2565", "#681f5e", "#711957", "#79134f", "#820c48", "#8b0641", "#94003a", "#8c0f42", "#841e4a", "#7b2e53", "#733d5b", "#6b4c63", "#635b6b", "#5b6a73", "#53797b", "#4a8984", "#42988c", "#3aa794", "#4caa92", "#5ead8f", "#70af8d", "#82b28b", "#94b588", "#a5b886", "#b7bb83", "#c9be81", "#dbc07f", "#edc37c", "#ffc67a"};
var colors = new string [] {
    "#D81B60", "#1E88E5", "#FFC107", "#004D40", "#c0e100", "#847b09", "#1c38ca", "#39d26d", "#6b16f8", "#55b765", "#fd5e8e", "#9f36c7", "#11beaa", "#119cab", "#bcf274", "#4aae8b", "#46801f", "#7f8f77", "#3d31a0", "#922993", "#2ceb38", "#970e95", "#f98bbd", "#ea74ac", "#b061a2", "#618066", "#98b3d1", "#1a630c", "#156f8b", "#14aeda", "#a8dd7a", "#fa123b", "#8a1fa1", "#a76c9b"
};
//List<Annotation> annotations = new ();
int colorIndx = 1;

int chartIndx = 1;
foreach(var g in locationDestributionOverTime[0].Data.GroupBy(x=>x["Description"])){
    // var axis = (new LinearAxis());
    // axis.SetValue("Title",g.Key.ToString());
    // axis.SetValue("showgrid", false);
    // axis.SetValue("showline", true);

    //annotations.Add(annotation);
    charts.Add(
        Chart2D.Chart.Column<string, int, string, string, string>(
            Name:g.Key.ToString()  ,
            keysValues: new List<Tuple<string,int>>(
                g.Select(kvp => new Tuple<string, int>(((DateTime)kvp["ReportDate"]).ToShortDateString(), int.Parse(kvp["BranchItemCount"].ToString())))
            )
            
        )
        .WithXAxisStyle(title: Title.init(),ShowGrid:false, ShowLine:true)
        //.WithYAxisStyle(title: Title.init("fasdfasdfasf"),ShowGrid:false, ShowLine:true, Id: StyleParam.SubPlotId.YAxis.NewYAxis(colorIndx), Side: StyleParam.Side.Right)
        //.WithYAxisStyle(title: Title.init(g.Key.ToString()),ShowGrid:false, ShowLine:false, Id: StyleParam.SubPlotId.YAxis.NewYAxis(colorIndx++), Side:StyleParam.Side.Left,Overlaying:StyleParam.LinearAxisId.NewY((colorIndx)))
        //.WithAnnotation(annotation)
        //.WithAxisAnchor(colorIndx++)
        //.WithLayout(Layout.init<int>(Width:1000))
        //.WithTitle(g.Key.ToString())
        //.WithXAxis(axis)
    );
}

var c1 = Color.fromColors(colors.Select(x=>Color.fromHex(x)));
var layout = Layout.init<int>(
    Width:2000,
    DragMode:StyleParam.DragMode.Pan,
    BarMode:StyleParam.BarMode.Stack
    ,
    //Colorway:Color.fromColors(new List<Color>(){Color.fromHex("#FFFFFF"), Color.fromHex("Fff111")})
    Colorway:Color.fromColors(colors.Select(x=>Color.fromHex(x)).ToList())
  //  ,Annotations:annotations
    // ,Annotations:new List<Annotation>() { 
    //     annotation                    
    //     //Annotation.init(1,1, Align:AnnotationAlignment.Center, ArrowColor:Color.fromString("black"),ArrowHead: ArrowHead.LineOnly, ArrowSide:ArrowSide.None, Text:"annotation")
    // }
    );
//Console.WriteLine(layoutGrid.TryGetValue("Rows"));
// Chart.Grid<IEnumerable<GenericChart.GenericChart>>(8,2)
// .Invoke(charts)
Chart.Combine(charts)
//.WithLayout(Layout.init<int>(Width:8000, Height:1500))
.WithLayout(layout
    )

Microsoft.DotNet.Interactive.Formatting.TabularData.TabularDataResource

<!-- Plotly chart will be drawn inside this DIV -->

https://peltiertech.com/stacked-bar-chart-alternatives/
Panel bar chart


In [ ]:
#!share --from sql-splSnapshots locationDestributionOverTime
using System;
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using Range = Plotly.NET.StyleParam.Range;
using static Plotly.NET.StyleParam;


List<GenericChart.GenericChart> charts = new();

var colors = new string [] {
    "#D81B60", "#1E88E5", "#FFC107", "#004D40", "#c0e100", "#847b09", "#1c38ca", "#39d26d", "#6b16f8", "#55b765", "#fd5e8e", "#9f36c7", "#11beaa", "#119cab", "#bcf274", "#4aae8b", "#46801", "#7f8f77", "#3d31a0", "#922993", "#2ceb38", "#970e95", "#f98bbd", "#ea74ac", "#b061a2", "#61806", "#98b3d1", "#1a630c", "#156f8b", "#14aeda", "#a8dd7a", "#fa123b", "#8a1fa1", "#a76c9b"
};

List<string> altLocations = new(){
    "Interlibrary Loan", "*** Temporary ***","Mobile Services", "Outreach", "Storage - Ask At Desk", "Unavailable: [do Not Use!!!]", "Wa Talking Book And Braille Library"
};
List<string> lowInventoryLocations = new(){
    "Outreach", "Storage - Ask At Desk", "Unavailable: [do Not Use!!!]", "Wa Talking Book And Braille Library","Interlibrary Loan"
};


var annotation = new Annotation();
int chartIndx = 1;
int highBranchRange = locationDestributionOverTime[0].Data.Where(x=>x["Description"].ToString() != "Central Library").Select(x=>int.Parse(x["BranchItemCount"].ToString())).Max();
int highCentralRange = locationDestributionOverTime[0].Data.Where(x=>x["Description"].ToString() == "Central Library").Select(x=>int.Parse(x["BranchItemCount"].ToString())).Max();
int highAltLocationsRange = locationDestributionOverTime[0].Data.Where(x=> altLocations.Contains(x["Description"].ToString())).Select(x=>int.Parse(x["BranchItemCount"].ToString())).Max();
int highLowInventoryLocationsRange = locationDestributionOverTime[0].Data.Where(x=> lowInventoryLocations.Contains(x["Description"].ToString())).Select(x=>int.Parse(x["BranchItemCount"].ToString())).Max();
foreach(var g in 
    locationDestributionOverTime[0]
    .Data
    .GroupBy(x => 
        x["Description"])
        .Where(x => 
            x.Key.ToString() != "Storage - Ask At Desk" //empty BranchItemCount
            )
            .OrderBy(x => 
                {
                    if(x.Key.ToString() == "Central Library"){  //force Central Library first
                        return 0;
                    }
                    else if(lowInventoryLocations.Contains(x.Key.ToString())){
                        return 3;
                    }
                    else if (altLocations.Contains(x.Key.ToString())){
                        return 2;
                    }else{
                        return 1;
                    }
                }).ThenBy(x=>x.Key.ToString()))
{
    annotation = new Annotation();
    annotation.SetValue("text", g.Key.ToString());
    annotation.SetValue("showarrow", false);
    annotation.SetValue("xref", $"x{chartIndx} domain" );
    annotation.SetValue("yref", $"y{chartIndx++} domain"); //paper
    annotation.SetValue("x", .5);
    annotation.SetValue("y", 1);
    annotation.SetValue("yshift", 30);
    
    int maxRange = 0;
    
    switch(g.Key.ToString()){
        case "Central Library":
            maxRange = highCentralRange;
            
            break;
        case string key when lowInventoryLocations.Contains(key):
            
            maxRange = highLowInventoryLocationsRange;
            break;    
        case string key when altLocations.Contains(key):
            maxRange = highAltLocationsRange;
            break;
       default:
            maxRange = highBranchRange;
            break;
    }
    charts.Add(
        Chart2D.Chart.Column<string, int, string, string, string>(
            Name:g.Key.ToString()  ,
            keysValues: new List<Tuple<string,int>>(
                g.Select(kvp => new Tuple<string, int>(((DateTime)kvp["ReportDate"]).ToShortDateString(), int.Parse(kvp["BranchItemCount"].ToString())))
            )   
        )
        .WithXAxisStyle(title: Title.init(),ShowGrid:false, ShowLine:true)
        .WithYAxisStyle(title: Title.init())
        .WithAnnotation(annotation)
        .WithYAxis(LinearAxis.init<int,int,int,int,int,int>(Range:Range.NewMinMax(0,maxRange*1.05)))
    );
}

var layout = Layout.init<int>(
    Width:1750,
     Height:5750,
     DragMode:StyleParam.DragMode.Pan
     ,Grid: LayoutGrid.init(
         SubPlots:
        new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[][]
        {
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(1), StyleParam.LinearAxisId.NewY(1)) ,  },
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(2), StyleParam.LinearAxisId.NewY(2)) ,Tuple.Create(StyleParam.LinearAxisId.NewX(3), StyleParam.LinearAxisId.NewY(3))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(4), StyleParam.LinearAxisId.NewY(4)) , Tuple.Create(StyleParam.LinearAxisId.NewX(5), StyleParam.LinearAxisId.NewY(5))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(6), StyleParam.LinearAxisId.NewY(6)) , Tuple.Create(StyleParam.LinearAxisId.NewX(7), StyleParam.LinearAxisId.NewY(7))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(8), StyleParam.LinearAxisId.NewY(8)) , Tuple.Create(StyleParam.LinearAxisId.NewX(9), StyleParam.LinearAxisId.NewY(9))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(10), StyleParam.LinearAxisId.NewY(10)) , Tuple.Create(StyleParam.LinearAxisId.NewX(11), StyleParam.LinearAxisId.NewY(11))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(12), StyleParam.LinearAxisId.NewY(12)) , Tuple.Create(StyleParam.LinearAxisId.NewX(13), StyleParam.LinearAxisId.NewY(13))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(14), StyleParam.LinearAxisId.NewY(14)) , Tuple.Create(StyleParam.LinearAxisId.NewX(15), StyleParam.LinearAxisId.NewY(15))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(16), StyleParam.LinearAxisId.NewY(16)) , Tuple.Create(StyleParam.LinearAxisId.NewX(17), StyleParam.LinearAxisId.NewY(17))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(18), StyleParam.LinearAxisId.NewY(18)) , Tuple.Create(StyleParam.LinearAxisId.NewX(19), StyleParam.LinearAxisId.NewY(19))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(20), StyleParam.LinearAxisId.NewY(20)) , Tuple.Create(StyleParam.LinearAxisId.NewX(21), StyleParam.LinearAxisId.NewY(21))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(22), StyleParam.LinearAxisId.NewY(22)) , Tuple.Create(StyleParam.LinearAxisId.NewX(23), StyleParam.LinearAxisId.NewY(23))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(24), StyleParam.LinearAxisId.NewY(24)) , Tuple.Create(StyleParam.LinearAxisId.NewX(25), StyleParam.LinearAxisId.NewY(25))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(26), StyleParam.LinearAxisId.NewY(26)) , Tuple.Create(StyleParam.LinearAxisId.NewX(27), StyleParam.LinearAxisId.NewY(27))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(28), StyleParam.LinearAxisId.NewY(28)) , Tuple.Create(StyleParam.LinearAxisId.NewX(29), StyleParam.LinearAxisId.NewY(29))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(30), StyleParam.LinearAxisId.NewY(30)) , Tuple.Create(StyleParam.LinearAxisId.NewX(31), StyleParam.LinearAxisId.NewY(31))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(32), StyleParam.LinearAxisId.NewY(32)) , Tuple.Create(StyleParam.LinearAxisId.NewX(33), StyleParam.LinearAxisId.NewY(33))}
           }
        )
     ,Colorway:Color.fromColors(
         Enumerable.Repeat(Color.fromKeyword(ColorKeyword.DeepSkyBlue),26)
         .Prepend(Color.fromKeyword(ColorKeyword.DarkBlue))
         .Concat(Enumerable.Repeat(Color.fromKeyword(ColorKeyword.Blueviolet),2))
         .Concat(Enumerable.Repeat(Color.fromKeyword(ColorKeyword.SteelBlue),4))
         )
     ,ShowLegend:false
    );
     
    
var g = Chart.Grid<IEnumerable<GenericChart.GenericChart>>(17,2
    ,YGap:.37    
    ,XGap:.1
    )
.Invoke(charts)
.WithLayout(layout
    );

var ax = GenericChart.getLayout(g).TryGetTypedValue<DynamicObj.DynamicObj>("xaxis");
ax.Value.SetValue("anchor","y");
ax.Value.SetValue("domain", new double[]{0,1});  //first chart 100% width
g

<!-- Plotly chart will be drawn inside this DIV -->